In [ ]:
!pip install focal-loss==0.0.2

In [ ]:
import pandas as pd
import numpy as np
import skopt
import gc
import time
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
import matplotlib.pyplot as plt
from skopt.plots import plot_convergence,plot_objective,plot_evaluations
import warnings
import random
import tensorflow as tf
from tensorflow.keras import Model
from focal_loss import BinaryFocalLoss
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv1D, Input, Dropout, BatchNormalization, MaxPooling1D, Flatten, Dense, GlobalMaxPooling1D
from skopt import dump, load
warnings.filterwarnings("ignore")

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
def pT_classes(x):
    if x<=25:
        return 0
    else:
        return 1

df = pd.read_csv('../input/cmsdata/CMS_trigger.csv').drop(columns=['Unnamed: 0'])
df['No. of hits'] = df[['Mask_'+str(i) for i in range(12)]].sum(axis = 1)
df = df[df['No. of hits']<9].reset_index(drop=True)
df['1/pT'] = df['q/pt'].abs()
df['pT'] = 1/df['1/pT']
df['pT_classes'] = df['pT'].apply(pT_classes)

features = list(np.array([['Phi_'+str(i),'Theta_'+str(i)] for i in range(12)]).reshape((-1)))
labels_1 = ['1/pT']
labels_2 = ['BendingAngle_'+str(i) for i in [0,1,9,10,11]]
labels_3 = ['pT_classes']
labels_4 = ['PatternStraightness']

scaler_1 = StandardScaler()
df[features] = scaler_1.fit_transform(df[features])
df[features] = df[features].fillna(0)

scaler_2 = MinMaxScaler()
df[labels_2] = scaler_2.fit_transform(df[labels_2])
df[labels_2] = df[labels_2].fillna(0)

scaler_3 = MinMaxScaler()
df[labels_4] = scaler_3.fit_transform(df[labels_4])
df[labels_4] = df[labels_4].fillna(0)

features = list(np.array([['Phi_'+str(i),'Theta_'+str(i), 'Mask_'+str(i)] for i in range(12)]).reshape((-1)))

df = df.sample(frac = 0.4).reset_index(drop = True)
df = df.iloc[:int((len(df)//1024)*1024)]

In [ ]:
X_train = df[features].iloc[:int(len(df)*0.8)].to_numpy().reshape((-1,12,3))
Y1_train = df[labels_1].iloc[:int(len(df)*0.8)]
Y2_train = df[labels_2].iloc[:int(len(df)*0.8)]
Y3_train = df[labels_3].astype('float32').iloc[:int(len(df)*0.8)]
Y4_train = df[labels_4].iloc[:int(len(df)*0.8)]

X_test = df[features].iloc[int(len(df)*0.8):].to_numpy().reshape((-1,12,3))
Y1_test = df[labels_1].iloc[int(len(df)*0.8):]
Y2_test = df[labels_2].iloc[int(len(df)*0.8):]
Y3_test = df[labels_3].astype('float32').iloc[int(len(df)*0.8):]
Y4_test = df[labels_4].iloc[int(len(df)*0.8):]

df = 0

In [ ]:
X_train.shape, len(Y1_train), len(Y2_train), len(Y3_train), len(Y4_train), X_test.shape, len(Y1_test)

In [ ]:
# Real - Dropout, Focal loss weight, SkipFraction
# Binary - Batchnorm, skipType
# Integer - number of layers, number of filters, focal loss gamma, number of dense neurons

In [ ]:
space  = [Real(0.0, 0.7, name='Dropout'),
          Real(0.0, 1.0, name='FocalLossWeight'),
          Real(0.0, 1.0, name='SkipFraction'),
          Categorical([0,1], name = 'BatchNorm'),
          Categorical([0,1], name = 'SkipType'),
          Integer(2, 10, name='# Layers'),
          Integer(16, 512, name='# Filters'),
          Integer(0, 5, name='FocalLossGamma'),
          Integer(128, 512, name='DenseNeurons'),
         ]

In [ ]:
def cnn(values = [0,1,0.5,1,1,5,16,2,128], X_train = X_train, Y1_train = Y1_train, Y2_train = Y2_train, Y3_train = Y3_train, Y4_train = Y4_train):
    global strategy
    dropout = values[0]
    FocalLossWeight = values[1]
    SkipFraction = values[2]
    batchNorm = values[3]
    SkipType = values[4]
    Layers = values[5]
    Filters = values[6]
    Gamma = values[7]
    DN = values[8]
    
    batch_size=1024* strategy.num_replicas_in_sync
    path = "model.h5"
    
    with strategy.scope():
        I = Input(shape=(12,3))
        x = Conv1D(filters=Filters, kernel_size=3, activation='relu', padding='same')(I)
        x_ = x
        for i in range(Layers-1):
            x = Dropout(dropout)(x)
            if batchNorm==1:
                x = BatchNormalization()(x)
            x__ = Conv1D(filters=Filters, kernel_size=3, activation='relu', padding='same')(x)
            if SkipType==1 and Layers*(1-SkipFraction)<=i:
                x = tf.concat([x_, x__] ,axis = -1)
                x_ = x__
            elif SkipType==0 and Layers*(1-SkipFraction)<=i:
                x = x_ + x__
                x_ = x__
            else:
                x_ = x__
                x = x__
        x = GlobalMaxPooling1D()(x)
        x1 = Dense(DN, activation='relu')(x)
        O1 = Dense(1, activation='sigmoid')(x1)
        x2 = Dense(DN, activation='relu')(x)
        O2 = Dense(5, activation='sigmoid')(x2)
        x3 = Dense(DN, activation='relu')(x)
        O3 = Dense(1, activation='sigmoid')(x3)
        x4 = Dense(DN, activation='relu')(x)
        O4 = Dense(1, activation='sigmoid')(x4)

        model = Model(inputs=I, outputs=[O1,O2, O3, O4])

        checkpoint = ModelCheckpoint(path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
        early_stop = EarlyStopping(monitor='val_loss',patience=3,verbose=0)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=0,verbose=0)

        model.compile(optimizer = 'adam', loss=['mse','mse',BinaryFocalLoss(gamma=Gamma),'mse'],loss_weights = [1,1,1,FocalLossWeight] )
    
#     model.summary()
#     print(X_train.shape, Y1_train.shape, Y2_train.shape, Y3_train.shape, Y4_train.shape )
    model.fit(x = X_train, y = [Y1_train, Y2_train, Y3_train, Y4_train], batch_size=batch_size, epochs=40, verbose=0, validation_split=0.1, callbacks=[checkpoint,early_stop,reduce_lr])
    
    model.load_weights(path)
    
    return model

In [ ]:
def score(true, preds):
    true = 1/true
    preds = 1/(preds+pow(10,-6))
    S = []
    count = 0
    for i in range(150):
        try:
            S.append(mae(true[(true>i)&(true<=i+1)],preds[(true>i)&(true<=i+1)])/(i+0.5))
        except:
            count+=1
            continue
    print(count)
    return sum(S)

In [ ]:
# def score(true, preds):
#     true = 1/true
#     preds = 1/(preds+pow(10,-6))
#     return mae(true,preds)

In [ ]:
iteration__ = 0
def objective(values = [0,1,0.5,1,1,5,16,2,128], X_test = X_test, Y1_test = Y1_test, Y2_test = Y3_test, Y3_test = Y3_test, Y4_test = Y4_test):
    global iteration__
    start = time.time()
    iteration__ += 1
    gc.collect()
    model = cnn(values)
    test_preds = model.predict(X_test)[0]
    if len(set(list(test_preds.reshape((-1)))))==1:
        model = cnn(values)
        test_preds = model.predict(X_test)[0]
    loss = score(Y1_test.to_numpy(), test_preds)
    print(iteration__, "iteration loss = ", loss)
    print('Time-taken = ', time.time()-start)
    print()
    return loss

In [ ]:
# x0 = []
# y0 = np.array([])
# for i in range(1,-1,-1):
#     res = load('../input/bayesian1/result'+str(i)+'.pkl')
#     x0 = x0 + res.x_iters
#     y0 = np.concatenate([y0,res.func_vals])
    
# for i in range(2):
#     res = load('../input/bayesian3/result'+str(i)+'.pkl')
#     x0 = x0 + res.x_iters
#     y0 = np.concatenate([y0,res.func_vals])

In [ ]:
# res_gp = gp_minimize(objective, space, n_calls=50, x0=x0, y0=y0, n_random_starts=5)

# "Best score=%.4f" % res_gp.fun

In [ ]:
res_gp = gp_minimize(objective, space, n_calls=40, n_random_starts=20)

"Best score=%.4f" % res_gp.fun

In [ ]:
x0 = res_gp.x_iters
y0 = res_gp.func_vals

for i in sorted(y0):
    print(i, x0[np.where(y0==i)[0][0]])

In [ ]:
dump(res_gp, 'result.pkl')

In [ ]:
print("Best parameters:")
print("- Dropout=",res_gp.x[0])
print("- FocalLossWeight=",res_gp.x[1])
print("- SkipFraction=",res_gp.x[2])
print("- BatchNorm=",res_gp.x[3])
print("- SkipType=",res_gp.x[4])
print("- # Layers=",res_gp.x[5])
print("- # Filters=",res_gp.x[6])
print("- FocalLossGamma=",res_gp.x[7])
print("- DenseNeurons=",res_gp.x[8])

In [ ]:
plot_convergence(res_gp)
plt.show()

In [ ]:
plot_objective(res_gp)
plt.show()

In [ ]:
plot_evaluations(res_gp)
plt.show()

In [ ]:
res_loaded = load('result.pkl')

In [ ]:
res_loaded.fun